# Dynamic lot-size model

In [ ]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt

## Formulation

$$
\begin{align}
    \text{min}~~ & \sum_{t \in T}{(h_{t} I_{t} + s_{t} y_{t})} \\
    \text{s.t.}~~ & I_{t} = I_{t - 1} + x_{t} - d_{t} & \forall ~ t \in T; t \geq 2\\
    & I_{1} = I_{0} + x_{1} - d_{1}\\
    & x_{t} \leq M y_{t} & \forall ~ t \in T \\
    & x_{t}; I_{t} \geq 0 & \forall ~ t \in T \\
    & y_{t} \in \left \{ 0, 1 \right \} & \forall ~ t \in T\\
\end{align}
$$

## Import data

In [ ]:
# Read input data
dataset = pd.read_csv("./input_wagner.csv", index_col=0)
dataset.head()

In [ ]:
# Obtain the maximum cost for comparison
max_cost = dataset.setup_cost.sum()
print(f"Maximum cost: {max_cost:.1f}")

## pyomo model

In [ ]:
# Initialize ConcreteModel
# model = ###

In [ ]:
# Set: Planning horizon
# model.T = pyo.Set(initialize=###)

In [ ]:
# Parameters
# ###.d = pyo.Param(###, initialize=###)
# model.s = pyo.Param(###, initialize=###)
# model.h = pyo.Param(###, initialize=###)

# Big M (can we do better?)
# model.M = pyo.Param(initialize=###)

In [ ]:
# Decision variables
# model.x = pyo.Var(###, within=###)
# model.y = pyo.Var(###, within=###)
# model.I = pyo.Var(###, within=###)

In [ ]:
# Inventory balance *special in the first instant
# def inventory_rule(model, t):
#     if t == model.T.first():
#         return ###
#     else:
#         t_prev = model.T.prev(t)
#         return ###


# model.inventory_rule = pyo.Constraint(###, rule=inventory_rule)

In [ ]:
# Indicator constraint activates y in case x is greater than zero
# def active_prod(###, ###):
#     return ###


# model.active_prod = pyo.Constraint(###, rule=active_prod)

In [ ]:
# Define the objective
# def total_holding(###):
#     return ###


# def total_setup(###):
#     return ###


# def total_cost(###):
#     return ###


# model.obj = pyo.Objective(rule=total_cost, sense=###)

## Solution

In [ ]:
solver = pyo.SolverFactory("appsi_highs")

In [ ]:
solver.solve(model, tee=True)

In [ ]:
opt_value = model.obj()
print(f"Best cost {opt_value}")
print(f"% savings {100 * (1 - opt_value / max_cost) :.2f}")

In [ ]:
dataset["production"] = [model.x[t].value for t in dataset.index]
dataset["inventory"] = [model.I[t].value for t in dataset.index]

In [ ]:
fig, ax = plt.subplots(figsize=[6, 3], dpi=100)
x = dataset.index
width = 0.35
ax.bar(x - width/2, dataset.production, width, color="darkgreen", label="production")
ax.bar(x + width/2, dataset.demand, width, color="navy", label="demand")
ax.set_xticks(x)
ax.set_ylabel("Qtd")
ax.set_xlabel("t")
ax.legend()
fig.tight_layout()
plt.show()